In [1]:
import pandas as pd
import numpy as np
import app_functions as af

In [3]:
data_dict = af.filter_data(r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\pivoted_data')[0]
weights_df = pd.DataFrame(dict(Factor = data_dict.keys(),Weight = [0,0,0,0,0.2,0.3,0,0,0.4,0,0,0,0.1],Type=[0,0,0,0,'bajo','alto',0,0,'bajo',0,0,0,'alto']))
n_quantiles = 10

In [4]:
weights_df = weights_df.loc[weights_df.Weight > 0]

In [5]:
weights_df

,Factor,Weight,Type
4,EVTOOperatingCashFlow,0.2,bajo
5,EVToSale,0.3,alto
8,PriceToCFPerShare,0.4,bajo
12,TotalDebtToEV,0.1,alto


In [129]:
def rank_data(pivoted_df, n_quantiles, type=['alto','bajo']):
    
    ranks_list = []
    labels = range(1,11) if type=='alto' else (range(10,0,-1) if type == 'bajo' else 'null')

    #### decil 0 tiene los valores mas bajos y el 9 los mas altos ####
    for row in pivoted_df.values:
        ranks_list.append(pd.qcut(row,n_quantiles,duplicates='drop',labels=labels))
    
    return pd.DataFrame(np.array(ranks_list), index=pivoted_df.index, columns=pivoted_df.columns)

In [101]:
list(range(10,0,-1)),list(range(1,11))

([10, 9, 8, 7, 6, 5, 4, 3, 2, 1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [51]:
tmp = data_dict['EV']

In [115]:
type='bajo'

labels = range(1,11) if type=='alto' else (range(10,0,-1) if type == 'bajo' else 'null')

In [114]:
labels

range(1, 11)

In [74]:
tmp

,1COV.DE,1U1.DE,A2.MI,A3M.MC,AAAA.L^C21,AAH.AS^D08,AAK.ST,AAL.L,AALB.AS,AB.L^G07,...,YAR.OL,YIT.HE,YSO.LS,ZALG.DE,ZEL.MC^K15,ZIGGO.AS^L14,ZO1G.DE^A22,ZODC.PA^C18,ZOT.MC^E22,ZURN.S
CallDate,,,,,,,,,,,,,,,,,,,,,
2001-02-01,NaN,NaN,5.067133e+09,NaN,NaN,4.083373e+10,NaN,2.628943e+10,NaN,5.326730e+09,...,NaN,NaN,3.678550e+09,NaN,2.995708e+09,NaN,NaN,NaN,NaN,5.137468e+10
2001-03-01,NaN,NaN,4.602721e+09,NaN,NaN,3.595008e+10,NaN,2.712922e+10,NaN,5.628050e+09,...,NaN,NaN,3.597966e+09,NaN,2.803983e+09,NaN,NaN,NaN,NaN,4.277390e+10
2001-04-01,NaN,NaN,4.690924e+09,NaN,NaN,3.114272e+10,NaN,2.934614e+10,NaN,5.664029e+09,...,NaN,NaN,3.507902e+09,NaN,2.245951e+09,NaN,NaN,1.512630e+09,1.466258e+09,3.043511e+10
2001-05-01,NaN,NaN,5.233800e+09,NaN,NaN,3.422244e+10,NaN,3.251516e+10,NaN,5.574082e+09,...,NaN,NaN,5.494462e+09,NaN,2.283896e+09,NaN,NaN,1.626490e+09,1.478257e+09,3.666960e+10
2001-06-01,NaN,NaN,5.149198e+09,NaN,NaN,1.059753e+11,NaN,3.227230e+10,NaN,5.295249e+09,...,NaN,NaN,5.234462e+09,NaN,2.287891e+09,NaN,NaN,1.754841e+09,1.481256e+09,3.467316e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,8.542856e+09,NaN,8.584196e+09,NaN,NaN,NaN,4.701924e+10,5.291988e+10,4.646394e+09,NaN,...,1.400725e+10,NaN,NaN,5.762853e+09,NaN,NaN,NaN,NaN,NaN,7.349125e+10
2022-10-01,8.535128e+09,NaN,8.270279e+09,NaN,NaN,NaN,4.410405e+10,5.240970e+10,4.435207e+09,NaN,...,1.243820e+10,NaN,NaN,5.283455e+09,NaN,NaN,NaN,NaN,NaN,6.890676e+10
2022-11-01,9.461276e+09,NaN,8.656253e+09,NaN,NaN,NaN,4.753485e+10,5.406793e+10,4.650650e+09,NaN,...,1.491977e+10,NaN,NaN,6.355850e+09,NaN,NaN,NaN,NaN,NaN,7.266846e+10


In [54]:
counter = 0
for i in tmp.values:
    counter+=1
counter

264

In [80]:
ranks_list = []

labels = np.array(range(0,n_quantiles))

#### decil 0 tiene los valores mas bajos y el 9 los mas altos ####
for row in tmp.values:
    ranks_list.append(pd.qcut(row,n_quantiles,duplicates='drop',labels=range(0,n_quantiles)))

In [88]:
pd.DataFrame(np.array(ranks_list),index=tmp.index,columns=tmp.columns)

,0,1,2,3,4,5,6,7,8,9,...,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394
0,NaN,NaN,4.0,NaN,NaN,8.0,NaN,8.0,NaN,4.0,...,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,8.0
1,NaN,NaN,4.0,NaN,NaN,8.0,NaN,8.0,NaN,4.0,...,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,8.0
2,NaN,NaN,4.0,NaN,NaN,8.0,NaN,8.0,NaN,4.0,...,NaN,NaN,3.0,NaN,2.0,NaN,NaN,1.0,1.0,8.0
3,NaN,NaN,4.0,NaN,NaN,8.0,NaN,8.0,NaN,4.0,...,NaN,NaN,4.0,NaN,2.0,NaN,NaN,1.0,1.0,8.0
4,NaN,NaN,4.0,NaN,NaN,9.0,NaN,8.0,NaN,4.0,...,NaN,NaN,4.0,NaN,2.0,NaN,NaN,1.0,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,3.0,NaN,3.0,NaN,NaN,NaN,7.0,7.0,1.0,NaN,...,4.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0
260,3.0,NaN,3.0,NaN,NaN,NaN,7.0,7.0,2.0,NaN,...,4.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0
261,3.0,NaN,3.0,NaN,NaN,NaN,7.0,7.0,1.0,NaN,...,5.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0
262,4.0,NaN,3.0,NaN,NaN,NaN,7.0,8.0,1.0,NaN,...,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,8.0


In [64]:
pd.qcut(data_dict['EV'].iloc[0],10,duplicates='drop',labels=range(0,10))

1COV.DE         NaN
1U1.DE          NaN
A2.MI             4
A3M.MC          NaN
AAAA.L^C21      NaN
               ... 
ZIGGO.AS^L14    NaN
ZO1G.DE^A22     NaN
ZODC.PA^C18     NaN
ZOT.MC^E22      NaN
ZURN.S            8
Name: 2001-02-01, Length: 1395, dtype: category
Categories (10, int64): [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9]

In [125]:
%%timeit
af.rank_data(data_dict['EV'],10)

229 ms ± 6.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
%%timeit
rank_data(data_dict['EV'],10,'bajo')

197 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
test_df.iloc[0].value_counts()

4.0    54
8.0    54
2.0    54
1.0    54
7.0    54
0.0    54
9.0    54
5.0    54
3.0    53
6.0    53
Name: 2001-02-01, dtype: int64

In [7]:
ranked_data_dict = {}
columns_list = []
for factor,type_ in zip(weights_df.Factor,weights_df.Type):
    ranked_data_dict[factor] = af.rank_data(data_dict[factor],n_quantiles,type_)[0]
    columns_list.append(set(ranked_data_dict[factor].columns))

In [8]:
columns_list

[{'SNEC.PA^B06',
  'NKT.CO',
  'PAGE.L',
  'MBTN.S',
  'ADYEN.AS',
  'CBRY.L^C10',
  'ICI.L^A08',
  'LISN.S',
  'GXIG.DE',
  'INTRUM.ST',
  'GTN.AS^C08',
  'THG.L',
  'BALN.S',
  'EPIRa.ST',
  'G24n.DE',
  'EGPW.MI^D16',
  'FCC.MC',
  'VATN.S',
  'LAGA.PA',
  'TTNr.AT^I19',
  'AROG.DE^C18',
  'EXOR.AS',
  'IASI.MI^A03',
  'WEIR.L',
  'ITPG.MI',
  'MOR.L^F10',
  'BION.S',
  'IMFI.VI',
  'SAB.L^J16',
  'MAP.MC',
  'ALIVsdb.ST',
  'FER.MC',
  'VPBN.S',
  'GPEG.L',
  'HAVA.PA^L17',
  'MTGb.ST',
  'ROG.S',
  'JYSK.CO',
  'SAABb.ST',
  'RTK.L^L02',
  'FNTGn.DE',
  'SAN.MC',
  'MICP.PA',
  'ELN.I^L13',
  'AWG.L^L06',
  'ICP.L',
  'ENGIE.PA',
  'SHB.L',
  'IETB.BR',
  'CEY.L',
  'ESNT.L',
  'BI.L^L06',
  'DEB.L^D19',
  'DNB.OL',
  'ENRO.ST',
  'BAB.L',
  'LCL.L^C18',
  'TOP.CO',
  'LHAG.DE',
  'VNUN.AS^G06',
  'PHNX.L',
  'CSR.L^H15',
  'HFGG.DE',
  'TNTE.AS^G16',
  'TLIT.MI',
  'AZN.L',
  'CPRI.MI',
  'GIVN.S',
  'GRLS.MC',
  'CGL.L^E15',
  'CLN.S',
  'CRH.I',
  'AEGS.L^C13',
  'MAB.L',
  'KA

In [5]:
common_columns = columns_list[0]
for i in columns_list[1:]:
    common_columns = common_columns & i
common_columns = list(common_columns)

In [6]:
weighted_df_dict = {}
for factor,ranked_df in ranked_data_dict.items():
    weight = float(weights_df.loc[weights_df.Factor==factor].Weight)
    weighted_df_dict[factor] = ranked_df[common_columns]*weight

In [7]:
dates = weighted_df_dict[list(weighted_df_dict.keys())[0]].index
final_df_list = []
for date in dates:
    summing_list = []
    try:
        for factor_df in weighted_df_dict.values():
            summing_list.append(factor_df.loc[date])
    except KeyError:
        continue
    final_df_list.append(sum(summing_list)/len(summing_list))

In [8]:
df = pd.DataFrame(final_df_list)

In [9]:
ranked_df = af.rank_data(df,n_quantiles)

In [10]:
rents_df = af.get_rents_df(ranked_df,r'C:\Users\hugo.perezdealbeniz\Desktop\Ranking DIP European Equities\ReutersEikon\data\Final Data\PriceClose.csv',n_quantiles)

In [11]:
colors = ["#0068c9","#d7dce6","#7f51b5","#ffd578","#ff902d","#8af0aa","#2db19f","#ffb1b1","#ff3030","#83c9ff",'#000000']

af.plot_NAV_absoluto(rents_df,colors)